# Imports

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_log_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder as SklearnOneHotEncoder

In [76]:
train_data = pd.read_csv("../../inputs/train/train.csv").drop("Id", axis=1)
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [77]:
test_data = pd.read_csv("../../inputs/test/test.csv")
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


# Removing outliers

In [78]:
train_data = train_data[train_data.GrLivArea < 4000]
train_data.shape

(1456, 80)

# Missing columns in train and test data

In [79]:
cols_with_missing_train = [col for col in train_data.columns
                     if train_data[col].isnull().any()]
print(cols_with_missing_train)

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [80]:
cols_with_missing_test = [col for col in test_data.columns
                     if test_data[col].isnull().any()]
print(cols_with_missing_test)

['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']


# Removing missing columns in train and test

In [81]:
train_data = train_data.drop(cols_with_missing_test, axis='columns')
train_data.shape

(1456, 47)

In [82]:
train_data.columns

Index(['MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond',
       'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [83]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 0 to 1459
Data columns (total 47 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSSubClass     1456 non-null   int64 
 1   LotArea        1456 non-null   int64 
 2   Street         1456 non-null   object
 3   LotShape       1456 non-null   object
 4   LandContour    1456 non-null   object
 5   LotConfig      1456 non-null   object
 6   LandSlope      1456 non-null   object
 7   Neighborhood   1456 non-null   object
 8   Condition1     1456 non-null   object
 9   Condition2     1456 non-null   object
 10  BldgType       1456 non-null   object
 11  HouseStyle     1456 non-null   object
 12  OverallQual    1456 non-null   int64 
 13  OverallCond    1456 non-null   int64 
 14  YearBuilt      1456 non-null   int64 
 15  YearRemodAdd   1456 non-null   int64 
 16  RoofStyle      1456 non-null   object
 17  RoofMatl       1456 non-null   object
 18  ExterQual      1456 non-null

In [84]:
test_data = test_data.drop(cols_with_missing_test, axis='columns')
test_data.shape

(1459, 47)

# Converting Categorical Variables to One-Hot Encoding

In [85]:
class OneHotEncoder(SklearnOneHotEncoder):
    def __init__(self, **kwargs):
        super(OneHotEncoder, self).__init__(**kwargs)
        self.fit_flag = False

    def fit(self, X, **kwargs):
        out = super().fit(X)
        self.fit_flag = True
        return out

    def transform(self, X, **kwargs):
        sparse_matrix = super(OneHotEncoder, self).transform(X)
        new_columns = self.get_new_columns(X=X)
        d_out = pd.DataFrame(
            sparse_matrix.toarray(), columns=new_columns, index=X.index
        )
        return d_out

    def fit_transform(self, X, **kwargs):
        self.fit(X)
        return self.transform(X)

    def get_new_columns(self, X):
        new_columns = []
        for i, column in enumerate(X.columns):
            j = 0
            while j < len(self.categories_[i]):
                new_columns.append(f"{column}_<{self.categories_[i][j]}>")
                j += 1
        return new_columns


def transform(data):
    cat_columns = data.select_dtypes(include=["object"]).columns
    cat_df = data[cat_columns]

    numeric_df = data.select_dtypes(include=np.number)

    ohe = OneHotEncoder(categories="auto", handle_unknown="ignore")
    ohe.fit(data[cat_columns])

    df_processed = ohe.transform(cat_df)

    df_processed_full = pd.concat([df_processed, numeric_df], axis=1)

    return df_processed_full

#     string_data = data.select_dtypes(include=['object'])
#     numeric_data = data.select_dtypes(include=np.number)
#     categorical_data = data[string_data]
#     ohe = OneHotEncoder(categories="auto", handle_unknown="ignore")
#     ohe.fit(data[string_data])
#     df_processed = ohe.transform(categorical_data)
#     df_processed_full = pd.concat([df_processed, numeric_data], axis=1)
#     return df_processed_full

train_final = transform(train_data)
test_final = transform(test_data)

print(train_final.shape, test_final.shape)

(1456, 155) (1459, 144)


In [86]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 0 to 1459
Columns: 155 entries, Street_<Grvl> to SalePrice
dtypes: float64(129), int64(26)
memory usage: 1.7 MB


In [87]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 144 entries, Street_<Grvl> to YrSold
dtypes: float64(118), int64(26)
memory usage: 1.6 MB


In [88]:
for column in train_final.columns:
    if column != 'SalePrice' and column not in test_final.columns:
        test_final[column] = 0.0
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 155 entries, Street_<Grvl> to Electrical_<nan>
dtypes: float64(129), int64(26)
memory usage: 1.7 MB


In [89]:
test_final.columns

Index(['Street_<Grvl>', 'Street_<Pave>', 'LotShape_<IR1>', 'LotShape_<IR2>',
       'LotShape_<IR3>', 'LotShape_<Reg>', 'LandContour_<Bnk>',
       'LandContour_<HLS>', 'LandContour_<Low>', 'LandContour_<Lvl>',
       ...
       'Condition2_<RRAn>', 'Condition2_<RRNn>', 'HouseStyle_<2.5Fin>',
       'RoofMatl_<Membran>', 'RoofMatl_<Metal>', 'RoofMatl_<Roll>',
       'Heating_<Floor>', 'Heating_<OthW>', 'Electrical_<Mix>',
       'Electrical_<nan>'],
      dtype='object', length=155)

# Using non-null 46 features to make predictions

In [90]:
final_train_data = train_final.drop('SalePrice', axis = 1)

# PCA to identify top features

In [91]:
pca = PCA()
pca.fit_transform(final_train_data)
pca.feature_names_in_

array(['Street_<Grvl>', 'Street_<Pave>', 'LotShape_<IR1>',
       'LotShape_<IR2>', 'LotShape_<IR3>', 'LotShape_<Reg>',
       'LandContour_<Bnk>', 'LandContour_<HLS>', 'LandContour_<Low>',
       'LandContour_<Lvl>', 'LotConfig_<Corner>', 'LotConfig_<CulDSac>',
       'LotConfig_<FR2>', 'LotConfig_<FR3>', 'LotConfig_<Inside>',
       'LandSlope_<Gtl>', 'LandSlope_<Mod>', 'LandSlope_<Sev>',
       'Neighborhood_<Blmngtn>', 'Neighborhood_<Blueste>',
       'Neighborhood_<BrDale>', 'Neighborhood_<BrkSide>',
       'Neighborhood_<ClearCr>', 'Neighborhood_<CollgCr>',
       'Neighborhood_<Crawfor>', 'Neighborhood_<Edwards>',
       'Neighborhood_<Gilbert>', 'Neighborhood_<IDOTRR>',
       'Neighborhood_<MeadowV>', 'Neighborhood_<Mitchel>',
       'Neighborhood_<NAmes>', 'Neighborhood_<NPkVill>',
       'Neighborhood_<NWAmes>', 'Neighborhood_<NoRidge>',
       'Neighborhood_<NridgHt>', 'Neighborhood_<OldTown>',
       'Neighborhood_<SWISU>', 'Neighborhood_<Sawyer>',
       'Neighborhood_<Sa

In [92]:
initial_feature_names = final_train_data.columns

In [93]:
most_important = [np.abs(pca.components_[i]).argmax() for i in range(30)]
most_important_names = [initial_feature_names[most_important[i]] for i in range(30)]
dic = {'PC{}'.format(i): most_important_names[i] for i in range(30)}
df1 = pd.DataFrame(dic.items())
df1

,0,1
0,PC0,LotArea
1,PC1,GrLivArea
2,PC2,MiscVal
3,PC3,1stFlrSF
4,PC4,WoodDeckSF
5,PC5,OpenPorchSF
6,PC6,OpenPorchSF
7,PC7,LowQualFinSF
8,PC8,ScreenPorch
9,PC9,MSSubClass


In [94]:
most_important = [np.abs(pca.components_[i]).argmax() for i in range(20)]
most_important_names = [initial_feature_names[most_important[i]] for i in range(20)]
dic = {'PC{}'.format(i): most_important_names[i] for i in range(20)}
df2 = pd.DataFrame(dic.items())
df2

,0,1
0,PC0,LotArea
1,PC1,GrLivArea
2,PC2,MiscVal
3,PC3,1stFlrSF
4,PC4,WoodDeckSF
5,PC5,OpenPorchSF
6,PC6,OpenPorchSF
7,PC7,LowQualFinSF
8,PC8,ScreenPorch
9,PC9,MSSubClass


In [95]:
train_1 = final_train_data[df1[1]]
train_1

,LotArea,GrLivArea,MiscVal,1stFlrSF,WoodDeckSF,OpenPorchSF,OpenPorchSF,LowQualFinSF,ScreenPorch,MSSubClass,...,Foundation_<CBlock>,Fireplaces,RoofStyle_<Gable>,LotConfig_<Inside>,BedroomAbvGr,HeatingQC_<Gd>,FullBath,ExterQual_<Gd>,ExterCond_<TA>,SaleCondition_<Normal>
0,8450,1710,0,856,0,61,61,0,0,60,...,0.0,0,1.0,1.0,3,0.0,2,1.0,1.0,1.0
1,9600,1262,0,1262,298,0,0,0,0,20,...,1.0,1,1.0,0.0,3,0.0,2,0.0,1.0,1.0
2,11250,1786,0,920,0,42,42,0,0,60,...,0.0,1,1.0,1.0,3,0.0,2,1.0,1.0,1.0
3,9550,1717,0,961,0,35,35,0,0,70,...,0.0,1,1.0,0.0,3,1.0,1,0.0,1.0,0.0
4,14260,2198,0,1145,192,84,84,0,0,60,...,0.0,1,1.0,0.0,4,0.0,2,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,1647,0,953,0,40,40,0,0,60,...,0.0,1,1.0,1.0,3,0.0,2,0.0,1.0,1.0
1456,13175,2073,0,2073,349,0,0,0,0,20,...,1.0,2,1.0,1.0,3,0.0,2,0.0,1.0,1.0
1457,9042,2340,2500,1188,0,60,60,0,0,70,...,0.0,2,1.0,1.0,4,0.0,2,0.0,0.0,1.0
1458,9717,1078,0,1078,366,0,0,0,0,20,...,1.0,0,0.0,1.0,2,1.0,1,0.0,1.0,1.0


In [96]:
train_2 = final_train_data[df2[1]]
train_2

,LotArea,GrLivArea,MiscVal,1stFlrSF,WoodDeckSF,OpenPorchSF,OpenPorchSF,LowQualFinSF,ScreenPorch,MSSubClass,PoolArea,3SsnPorch,YearBuilt,YearRemodAdd,MoSold,YrSold,TotRmsAbvGrd,OverallCond,OverallQual,LotShape_<Reg>
0,8450,1710,0,856,0,61,61,0,0,60,0,0,2003,2003,2,2008,8,5,7,1.0
1,9600,1262,0,1262,298,0,0,0,0,20,0,0,1976,1976,5,2007,6,8,6,1.0
2,11250,1786,0,920,0,42,42,0,0,60,0,0,2001,2002,9,2008,6,5,7,0.0
3,9550,1717,0,961,0,35,35,0,0,70,0,0,1915,1970,2,2006,7,5,7,0.0
4,14260,2198,0,1145,192,84,84,0,0,60,0,0,2000,2000,12,2008,9,5,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,1647,0,953,0,40,40,0,0,60,0,0,1999,2000,8,2007,7,5,6,1.0
1456,13175,2073,0,2073,349,0,0,0,0,20,0,0,1978,1988,2,2010,7,6,6,1.0
1457,9042,2340,2500,1188,0,60,60,0,0,70,0,0,1941,2006,5,2010,9,9,7,1.0
1458,9717,1078,0,1078,366,0,0,0,0,20,0,0,1950,1996,4,2010,5,6,5,1.0


# PCA top 30 features

## Batch Size 50

In [97]:
model1 = MLPRegressor(solver='adam', early_stopping = True, hidden_layer_sizes=(100, 20), random_state=1, max_iter = 2000, batch_size=50, shuffle=True)
model1.fit(train_1.values ,train_final['SalePrice'].values)

MLPRegressor(batch_size=50, early_stopping=True, hidden_layer_sizes=(100, 20),
             max_iter=2000, random_state=1)

In [98]:
pred = model1.predict(test_final[train_1.columns].values)
output = pd.DataFrame(columns=['Id', 'SalePrice'])
output['Id'] = test_data['Id']
output['SalePrice'] = pred

np.unique(output['SalePrice'], return_index=False, return_inverse=False, return_counts=True, axis=None)
output.to_csv("../../outputs/pca_30_config1.csv", index=None)

## Batch Size 200

In [99]:
model2 = MLPRegressor(solver='adam', early_stopping = True, hidden_layer_sizes=(100, 20), random_state=1, max_iter = 2000, batch_size=200, shuffle=True)
model2.fit(train_1.values ,train_final['SalePrice'].values)

MLPRegressor(batch_size=200, early_stopping=True, hidden_layer_sizes=(100, 20),
             max_iter=2000, random_state=1)

In [100]:
pred = model2.predict(test_final[train_1.columns].values)
output = pd.DataFrame(columns=['Id', 'SalePrice'])
output['Id'] = test_data['Id']
output['SalePrice'] = pred

np.unique(output['SalePrice'], return_index=False, return_inverse=False, return_counts=True, axis=None)
output.to_csv("../../outputs/pca_30_config2.csv", index=None)

# PCA top 20 features

## Batch size - 50

In [101]:
model3 = MLPRegressor(solver='adam', early_stopping = True, hidden_layer_sizes=(100, 20), random_state=1, max_iter = 2000, batch_size=50, shuffle=True)
model3.fit(train_2.values ,train_final['SalePrice'].values)

MLPRegressor(batch_size=50, early_stopping=True, hidden_layer_sizes=(100, 20),
             max_iter=2000, random_state=1)

In [102]:
pred = model3.predict(test_final[train_2.columns].values)
output = pd.DataFrame(columns=['Id', 'SalePrice'])
output['Id'] = test_data['Id']
output['SalePrice'] = pred

np.unique(output['SalePrice'], return_index=False, return_inverse=False, return_counts=True, axis=None)
output.to_csv("../../outputs/pca_20_config1.csv", index=None)

## Batch size - 200

In [103]:
model4 = MLPRegressor(solver='adam', early_stopping = True, hidden_layer_sizes=(100, 20), random_state=1, max_iter = 2000, batch_size=200, shuffle=True)
model4.fit(train_2.values ,train_final['SalePrice'].values)

MLPRegressor(batch_size=200, early_stopping=True, hidden_layer_sizes=(100, 20),
             max_iter=2000, random_state=1)

In [104]:
pred = model4.predict(test_final[train_2.columns].values)
output = pd.DataFrame(columns=['Id', 'SalePrice'])
output['Id'] = test_data['Id']
output['SalePrice'] = pred

np.unique(output['SalePrice'], return_index=False, return_inverse=False, return_counts=True, axis=None)
output.to_csv("../../outputs/pca_20_config2.csv", index=None)